# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *
from qiskit.providers.ibmq.managed import IBMQJobManager

In [2]:
import rotcounterrot as rcr
import vqe

# Setup

In [3]:
token = ''  # Put in your IBM token
IBMQ.save_account(token, overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(group='open')

In [4]:
backend_sim = 'ibmq_qasm_simulator'
backends = ["ibmq_manila", "ibmq_quito", "ibmq_belem", "ibmq_lima", "ibmq_oslo", "ibmq_nairobi"]

job_manager = IBMQJobManager()
backend = provider.get_backend(backend_sim) # Either use backend_sim for simulator, or backends[x] for device
print(backend)

ibmq_qasm_simulator


# Running the Rotation-Counter Rotation Experiment

In [5]:
# Coefficients of T operator
acoeff_ar = np.load("")  # or np.genfromtxt("")
bcoeff_ar = np.load("")  # or np.genfromtxt("")

In [6]:
# Cell for reducing number of points for quicker tests
# start:stop:step
acoeff_ar = acoeff_ar[0:200:200]
bcoeff_ar = bcoeff_ar[0:200:200]

In [7]:
best_qubits = [0, 1, 2, 3]  # A list of four (linearly connected) qubits
shots_num = 8000
beta = np.pi/2

### Getting data without measurement error mitigation

In [9]:
logicXavg_ar, logicXerr_ar = rcr.many_exp_point(backend, best_qubits, shots_num, job_manager, beta, acoeff_ar, bcoeff_ar)

Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.


### Getting data with measurement error mitigation

In [16]:
meas_filter = rcr.run_meas_error_calibs(backend, best_qubits, shots_num)

Job Status: job has successfully run


In [12]:
logicXavg_ar, logicXerr_ar = rcr.many_exp_point(backend, best_qubits, shots_num, job_manager, beta, acoeff_ar, bcoeff_ar, meas_filter)

Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.


### Saving data

In [ ]:
np.savetxt("", logicXavg_ar, delimiter=",")
np.savetxt("", logicXerr_ar, delimiter=",")

# Plotting

In [2]:
alpha_ar = np.linspace(0, np.pi/2, len(acoeff_ar))

In [ ]:
other_data = np.genfromtxt('', delimiter=',')

In [ ]:
plt.errorbar(alpha_ar, logicXavg_ar, yerr = 2*np.array(logicXerr_ar), fmt = "o",color = "blue", markersize=3, label = "Circuit Result")
# plt.plot(alpha_ar, other_data, "r-", label="Theory result")
plt.legend()
plt.title(r"Logical <$\overline{X}$> vs. Sweep Parameter", size = 16)
plt.xlabel(r"Sweep parameter $\frac{2\alpha}{\pi}$", size = 12)
plt.ylabel(r"<$\overline{X}$> = <$(-1)^{s_1 + s_3}$>", size = 12)
# plt.savefig()
plt.show()

# Running VQE

In [5]:
theta_ar = np.linspace(0, np.pi/4, 100)

Note: If just using the raw VQE data, then only $0 \leq \theta \leq \pi/4$ is required. If planning to do Fourier sum fits to the VQE data, then $0 \leq \theta \leq \pi/2$ for $\langle K_i \rangle$ and $0 \leq \theta \leq \pi$ for $\langle X_i \rangle$ should be used to capture a full period.

In [6]:
best_qubits_one = [0]  # A list containing the position of the best single qubit
best_qubits_three = [0,1,2]  # A list containing the postion of the best linearly connected three qubits
shots_num = 8000

### Running VQE without measurment error mitigation

In [7]:
Xavg_ar, Xerr_ar = vqe.multiple_avgX(backend, best_qubits_one, shots_num, job_manager, theta_ar)

Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.


In [8]:
Kavg_ar, Kerr_ar = vqe.multiple_avgK(backend, best_qubits_one, best_qubits_three, shots_num, job_manager, theta_ar)

Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.


### Running VQE with measurement error mitigation

In [7]:
meas_filter_one = rcr.run_meas_error_calibs(backend, best_qubits_one, shots_num)

Job Status: job has successfully run


In [11]:
meas_filter_three = rcr.run_meas_error_calibs(backend, best_qubits_three, shots_num)

Job Status: job has successfully run


In [8]:
Xavg_ar, Xerr_ar = vqe.multiple_avgX(backend, best_qubits_one, shots_num, job_manager, theta_ar, meas_filter_one)

Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.
Job Status: job has successfully run
All jobs have finished.


In [ ]:
Kavg_ar, Kerr_ar = vqe.multiple_avgK(backend, best_qubits_one, best_qubits_three, shots_num, job_manager, theta_ar, meas_filter_one, meas_filter_three)

### Saving data

In [ ]:
np.savetxt("", Xavg_ar, delimiter=",")
np.savetxt("", Xerr_ar, delimiter=",")
np.savetxt("", Kavg_ar, delimiter=",")
np.savetxt("", Kerr_ar, delimiter=",")

# Getting Coefficients from VQE

In [8]:
alpha_ar = np.linspace(0, np.pi/2, 100)

### Direct Calculation

In [9]:
thetamin_raw_ar = vqe.multiple_get_thetamin_VQE(alpha_ar, theta_ar, Xavg_ar, Kavg_ar)
acoeff_ar, bcoeff_ar = vqe.getT_VQE(thetamin_raw_ar)

NameError: name 'alpha_ar' is not defined

### Calculation with Fourier Fits

In [9]:
maxn = 50

In [10]:
Xfouriercoeff_ar = vqe.mult_fouriercoeff(theta_ar, np.pi, maxn, Xavg_ar)
Kfouriercoeff_ar = vqe.mult_fouriercoeff(theta_ar, np.pi/2, maxn, Kavg_ar)

In [11]:
Xapprox_ar = vqe.fourierfit(theta_ar, [1, 3, 5, 7, 9], Xfouriercoeff_ar)
Kapprox_ar = vqe.fourierfit(theta_ar, [0, 2, 4, 6, 8], Kfouriercoeff_ar)#

Note that $\langle X_i \rangle$ is an odd function of $\theta$ and $\langle K_i \rangle$ is an even function of $\theta$. So, when calculating the fourier sum approximation, we can be selective with the coefficients. Also, the fourier sum is better truncated before the coeffficients get too high, wherin the fit will add back the experimental noise (of which the removal is the entire point of the fourier fitting). A plot of the magnitude of the coefficients can be done to see at what point they drop off and hence can be neglected in the fit.

In [ ]:
thetamin_approx_ar = vqe.multiple_get_thetamin_VQE(alpha_ar, theta_ar, Xapprox_ar, Kapprox_ar)
acoeff_ar, bcoeff_ar = vqe.getT_VQE(thetamin_ar)

### Analytic Result

In [9]:
Xavg_analytic_ar = vqe.predictedX(theta_ar)
Kavg_analytic_ar = vqe.predictedK(theta_ar)

thetamin_analytic_ar = vqe.multiple_get_thetamin_VQE(alpha_ar, theta_ar, Xavg_analytic_ar, Kavg_analytic_ar)
acoeff_ar, bcoeff_ar = vqe.getT_VQE(thetamin_analytic_ar)

[0.         0.00793323 0.00793323 0.01586596 0.01586596 0.0237977
 0.0237977  0.03172793 0.03172793 0.03965617 0.03965617 0.04758192
 0.04758192 0.05550466 0.05550466 0.06342392 0.06342392 0.07133918
 0.07133918 0.07924996 0.07924996 0.08715574 0.08715574 0.09505604
 0.09505604 0.10295036 0.10295036 0.1108382  0.1108382  0.11871906
 0.11871906 0.12659245 0.13445788 0.13445788 0.14231484 0.14231484
 0.15016284 0.15016284 0.1580014  0.16583001 0.16583001 0.17364818
 0.17364818 0.18145542 0.18145542 0.18925124 0.19703516 0.19703516
 0.20480667 0.21256529 0.21256529 0.22031053 0.22804191 0.22804191
 0.23575894 0.24346112 0.24346112 0.25114799 0.25881905 0.25881905
 0.26647381 0.27411181 0.28173256 0.28173256 0.28933557 0.29692038
 0.30448649 0.31203345 0.31203345 0.31956076 0.32706796 0.33455458
 0.34202014 0.34946418 0.35688622 0.3642858  0.37166246 0.37901572
 0.38634513 0.39365022 0.40093054 0.40818562 0.42261826 0.42979491
 0.43694451 0.44406661 0.45822652 0.46526344 0.47924899 0.48619

## Saving Results

In [ ]:
np.savetxt("", acoeff_ar, delimiter=",")
np.savetxt("", bcoeff_ar, delimiter=",")